In [110]:
import os
import mne
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
from mne.time_frequency import psd_array_multitaper

In [111]:
# Path to the dataset root
data_dir = './ds003523'

# Initialize lists for features and labels
all_features = []
all_labels = []

# Define frequency bands
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}


# Load participants.tsv to get the Group labels
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')



In [ ]:
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')
participants

In [ ]:
participants.Group.value_counts()

### Helper Functions

In [114]:
# Function to compute additional features
def compute_additional_features(epoch_data):
    """
    Computes additional features like:
    - Mean, variance, skewness, and kurtosis for time-domain data
    - Line length (signal complexity)
    """
    from scipy.stats import skew, kurtosis

    # Mean and variance across time points
    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)

    # Skewness and kurtosis across time points
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)

    # Line length (sum of absolute differences between consecutive samples)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

In [115]:
# X = np.vstack(all_features)  # Features for all subjects and sessions
# y = np.hstack(all_labels)  

In [ ]:
# Function to compute additional features (mean, variance, skewness, kurtosis, line length)
def compute_additional_features(epoch_data):
    if len(epoch_data.shape) != 2:
        raise ValueError(f"Expected 2D array for epoch_data, got {epoch_data.shape}")

    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

# Initialize variables
all_features = []
all_labels = []
all_subjects = []  # To store subject IDs for each epoch
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}
expected_feature_count = 16  # Total number of features (5 frequency + 5 time-domain)

# Loop through each subject
    # Loop through each subject
for subject in range(0, 10):  # Adjust range as needed
    subject_id = f"sub-{subject:03d}"  # Format: sub-001, sub-002, etc.
    subject_id_lower = subject_id.lower()
    
    # Condition to skip a specific subject
    if subject_id_lower == "sub-056":
        continue  # Skip this iteration
    # Condition to skip a specific subject
    if subject_id_lower == "sub-077":
        continue  # Skip this iteration
    # Condition to skip a specific subject
    if subject_id_lower == "sub-079":
        continue  # Skip this iteration

    # Your code for processing other subjects
    print(f"Processing {subject_id_lower}")

    # Check if subject exists in participants.tsv
    if subject_id_lower not in participants['participant_id'].values:
        print(f"Subject {subject_id} not found in participants.tsv. Skipping.")
        continue

    # Locate the subject directory
    subject_dir = os.path.join(data_dir, subject_id)
    if not os.path.exists(subject_dir):
        print(f"Subject directory {subject_id} not found. Skipping.")
        continue

    # Find all available sessions for the subject
    #sessions = [s for s in os.listdir(subject_dir) if s.startswith('ses-')]
    for session in ["ses-01"]:
        session_path = os.path.join(subject_dir, session, 'eeg')




    for session in ["ses-01"]:
        session_path = os.path.join(subject_dir, session, 'eeg')

        # Retrieve session data using DataLad
        os.system(f'datalad get {session_path}')

        # Check if the .set file exists for this session
        eeg_file = os.path.join(session_path, f"{subject_id}_{session}_task-VisualWorkingMemory_eeg.set")
        if not os.path.exists(eeg_file):
            print(f"Data for {subject_id}, {session} not found. Skipping.")
            continue

        # Load the EEG data
        raw = mne.io.read_raw_eeglab(eeg_file, preload=True)

        # Preprocess the data
        raw.filter(0.5, 100., fir_design='firwin')

        # Extract events
        events, event_id = mne.events_from_annotations(raw)

        # Define epochs
        tmin, tmax = -0.2, 0.5
        epochs = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True)

        # Compute PSD for the entire frequency range
        psd = epochs.compute_psd(method='multitaper', fmin=0.5, fmax=100)
        psd_data = psd.get_data()  # Shape: (n_epochs, n_channels, n_freqs)
        freqs = psd.freqs

        # Compute features for each frequency band
        band_powers = {}
        for band, (fmin, fmax) in band_ranges.items():
            band_indices = (freqs >= fmin) & (freqs < fmax)
            band_power = psd_data[:, :, band_indices].mean(axis=2)  # Mean PSD within the band
            band_powers[band] = band_power.mean(axis=1)  # Aggregate across channels

        # Extract time-domain data from epochs
        epoch_data = epochs.get_data().mean(axis=1)  # Aggregate across channels (shape: n_epochs, n_times)

        

In [ ]:
import numpy as np

# Step 1: Check Available Channels
print("Available channels:", epochs.ch_names)

# Step 2: Select & Extract Data for Specific Channels
channels_to_extract = ["AFz", "FT8", "FC2", "PO8", "O2", "PO7"]

# Ensure the selected channels exist in the dataset
channel_indices = [epochs.ch_names.index(ch) for ch in channels_to_extract if ch in epochs.ch_names]

print("Selected channel indices:", channel_indices)
print("Selected channel names:", [epochs.ch_names[i] for i in channel_indices])

# Extract waveform data for selected channels
waveform_data = epochs.get_data()[:, channel_indices, :]  # Shape: (n_epochs, n_selected_channels, n_times)

# Step 3: Compute Features (Mean Over Time)
waveform_features = waveform_data.mean(axis=2)  # Shape: (n_epochs, n_selected_channels)

# Step 4: Ensure `all_features` Includes Channels
if 'all_features' not in locals():  # Check if all_features exists
    all_features = np.zeros((waveform_features.shape[0], 0))  # Initialize with correct row count if empty
# Initialize all_features if it's empty
all_features = np.zeros((waveform_features.shape[0], 0))  # Initialize with correct row count if empty

# Check if all_features is 1D and reshape it temporarily
if all_features.ndim == 1:
    all_features = all_features[:, np.newaxis]  # Make it 2D if it's 1D

# Concatenate waveform features
all_features = np.hstack((all_features, waveform_features))

# Store feature names (optional, for tracking)
feature_channel_names = ["Original_Features"] + channels_to_extract
# Concatenate waveform features
all_features = np.hstack((all_features, waveform_features))

# Store feature names (optional, for tracking)
feature_channel_names = ["Original_Features"] + channels_to_extract

# Step 5: Final Verification
print("Final feature shape:", all_features.shape)
print("Feature labels:", feature_channel_names)

In [123]:

# Initialize as empty lists, NOT NumPy arrays
all_features = []
all_subjects = []
all_labels = []

for session in ["ses-01"]:  # Or for participant in participants if needed
    # Compute additional features
    mean_values, var_values, skew_values, kurtosis_values, line_length = compute_additional_features(epoch_data)

    # Combine all features into a single array
    try:
        session_features = np.column_stack([
            band_powers['delta'], band_powers['theta'], band_powers['alpha'],
            band_powers['beta'], band_powers['gamma'], mean_values,
            var_values, skew_values, kurtosis_values, line_length,
            "AFz", "FT8", "FC2", "PO8", "O2", "PO7"
        ])
        print(f"Session features shape for {subject_id}, {session}: {session_features.shape}")
    except ValueError as e:
        print(f"Feature dimension mismatch for {subject_id}, {session}: {e}")
        continue  # This will now work as it's inside the loop

    # Append subject ID for each epoch
    subject_ids = np.repeat(subject_id, session_features.shape[0])

    all_features.append(session_features)
    all_subjects.append(subject_ids)

    # Get the group label for the subject
    group_label = participants.loc[participants['participant_id'] == subject_id_lower, 'Group'].values
    if len(group_label) == 0 or pd.isna(group_label[0]):
        print(f"No valid group label found for {subject_id}. Skipping.")
        continue  # This will also work here

    # Repeat the label for each epoch in the session
    all_labels.append(np.repeat(group_label[0], session_features.shape[0]))

Feature dimension mismatch for sub-009, ses-01: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 510 and the array at index 10 has size 1


In [126]:
# Compute additional features
mean_values, var_values, skew_values, kurtosis_values, line_length = compute_additional_features(epoch_data)

# Ensure `waveform_features` is included
session_features = np.column_stack([
    waveform_features,  # EEG waveform-based features (6 channels)
    band_powers['delta'], band_powers['theta'], band_powers['alpha'],
    band_powers['beta'], band_powers['gamma'], mean_values,
    var_values, skew_values, kurtosis_values, line_length,
     "AFz", "FT8", "FC2", "PO8", "O2", "PO7" # Other computed features
])

# Check if feature shape is correct
print(f"Session features shape for {subject_id}, {session}: {session_features.shape}")  # Should be (510, 16)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 510 and the array at index 11 has size 1

In [125]:
training_data.head(100000)

,delta,theta,alpha,beta,gamma,mean,variance,skewness,kurtosis,line_length,label
0,0.000123,0.000067,1.039174e-05,4.101177e-07,1.740069e-07,-0.000003,2.089062e-08,-1.763948,2.470575,0.004479,0
1,0.000208,0.000085,9.628139e-07,2.513081e-07,1.123781e-07,0.000125,4.599112e-08,0.497322,-1.170764,0.003333,0
2,0.000917,0.000410,3.276360e-05,4.333310e-06,3.227772e-07,0.000167,1.393151e-07,0.577952,-0.144130,0.005464,0
3,0.000069,0.000030,1.999117e-06,3.742573e-07,1.669777e-07,-0.000084,1.022358e-08,0.009396,-1.516320,0.003864,0
4,0.000231,0.000103,7.266782e-06,3.089532e-06,2.499983e-07,0.000274,4.820566e-08,-0.132097,-0.538502,0.004247,0
...,...,...,...,...,...,...,...,...,...,...,...
4575,0.000007,0.000009,1.116148e-05,6.411663e-06,1.339507e-05,-0.000019,3.349408e-08,-0.474056,0.315111,0.038194,1
4576,0.000038,0.000018,7.063376e-06,3.822359e-06,1.338066e-05,-0.000027,3.793034e-08,0.057487,-0.799786,0.038020,1
4577,0.000010,0.000009,7.203328e-06,4.143663e-06,1.342848e-05,0.000018,3.252372e-08,-0.229151,-0.742725,0.038347,1
4578,0.000017,0.000013,8.710416e-06,4.783456e-06,1.329040e-05,-0.000026,3.571327e-08,-0.222827,-0.480546,0.038169,1
